In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
import zipfile
from zipfile import BadZipFile
from datetime import datetime

In [2]:
caminho = Path("dados_psat")
caminhos_zip = list(caminho.rglob("*.zip"))
caminhos_txt = list(caminho.rglob("*.txt"))

## Descompactar Zip

In [5]:
for psat_zip in caminhos_zip:
    try:
        with zipfile.ZipFile(psat_zip) as zip:
            zip.extractall(caminho)
            psat_zip.unlink()
            
    except BadZipFile as e:
        print(f"{psat_zip} não é zip!")

dados_psat/psath_25042022.zip não é zip!
dados_psat/psath_09092021.zip não é zip!
dados_psat/psath_08092020.zip não é zip!
dados_psat/psath_16052023.zip não é zip!


## Transformar os psats em dataframe

In [9]:
lista_df = []

for caminho in caminhos_txt:
    nome_arquivo = caminho.stem
    data_arquivo = nome_arquivo.split("_")[-1]
    data = datetime.strptime(data_arquivo, "%d%m%Y")
    
    df = pd.read_csv(caminho, sep=r"\s+", names=["PSAT", "LAT", "LON", "CHUVA"])
    df["DATA"] = data
    df.set_index("DATA", inplace=True)

    lista_df.append(df)

df_psats = pd.concat(lista_df, axis=0).sort_index()

df_psats

,PSAT,LAT,LON,CHUVA
DATA,,,,
2019-08-14,PSATCNV,-27.60,-51.33,0.02
2019-08-14,PSATSME,-13.40,-48.30,0.00
2019-08-14,PSATNAV,-21.12,-50.20,0.01
2019-08-14,PSATIBT,-21.76,-48.99,0.06
2019-08-14,PSATESZ,-23.46,-46.91,0.52
...,...,...,...,...
2025-05-13,PSATCBIV,-16.15,-48.67,0.00
2025-05-13,PSATCBI,-17.99,-48.54,0.02
2025-05-13,PSATPTQ,-23.18,-53.17,0.00


In [10]:
df_psats.to_csv("historico_psat.csv")

In [11]:
for data in df_psats.index.unique():
    print(data)

2019-08-14 00:00:00
2019-08-15 00:00:00
2019-08-16 00:00:00
2019-08-17 00:00:00
2019-08-18 00:00:00
2019-08-19 00:00:00
2019-08-20 00:00:00
2019-08-21 00:00:00
2019-08-22 00:00:00
2019-08-23 00:00:00
2019-08-24 00:00:00
2019-08-25 00:00:00
2019-08-26 00:00:00
2019-08-27 00:00:00
2019-08-28 00:00:00
2019-08-29 00:00:00
2019-08-30 00:00:00
2019-08-31 00:00:00
2019-09-01 00:00:00
2019-09-02 00:00:00
2019-09-03 00:00:00
2019-09-04 00:00:00
2019-09-05 00:00:00
2019-09-06 00:00:00
2019-09-07 00:00:00
2019-09-08 00:00:00
2019-09-09 00:00:00
2019-09-10 00:00:00
2019-09-11 00:00:00
2019-09-12 00:00:00
2019-09-13 00:00:00
2019-09-14 00:00:00
2019-09-15 00:00:00
2019-09-16 00:00:00
2019-09-17 00:00:00
2019-09-18 00:00:00
2019-09-19 00:00:00
2019-09-20 00:00:00
2019-09-21 00:00:00
2019-09-22 00:00:00
2019-09-23 00:00:00
2019-09-24 00:00:00
2019-09-25 00:00:00
2019-09-26 00:00:00
2019-09-27 00:00:00
2019-09-28 00:00:00
2019-09-29 00:00:00
2019-09-30 00:00:00
2019-10-01 00:00:00
2019-10-02 00:00:00


## Localizando ponto geográfico dos PSATS

In [3]:
psats_df = pd.read_csv("historico_psat.csv")
psats_df

,DATA,PSAT,LAT,LON,CHUVA
0,2019-08-14,PSATCNV,-27.60,-51.33,0.02
1,2019-08-14,PSATSME,-13.40,-48.30,0.00
2,2019-08-14,PSATNAV,-21.12,-50.20,0.01
3,2019-08-14,PSATIBT,-21.76,-48.99,0.06
4,2019-08-14,PSATESZ,-23.46,-46.91,0.52
...,...,...,...,...,...
213633,2025-05-13,PSATCBIV,-16.15,-48.67,0.00
213634,2025-05-13,PSATCBI,-17.99,-48.54,0.02
213635,2025-05-13,PSATPTQ,-23.18,-53.17,0.00
213636,2025-05-13,PSATCPV,-22.65,-51.33,0.01


In [ ]:
geometria = [
        Point(xy) for xy in zip(psats_df["LON"], psats_df["LAT"])
    ]
gdf_psat = gpd.GeoDataFrame(
        psats_df, geometry=geometria, crs="EPSG:4326"
    )

gdf_psat.to_csv("historico_psat_ponto.csv")